In [1]:
from __future__ import print_function

from keras.models import Sequential, Model
from keras.layers.embeddings import Embedding
from keras.layers import Input, Activation, Dense, Permute, Dropout, add, dot, concatenate
from keras.layers import LSTM
from keras.utils.data_utils import get_file
from keras.preprocessing.sequence import pad_sequences
from functools import reduce
import tarfile
import numpy as np
import re

Using TensorFlow backend.


In [2]:
def tokenize(sent):
    '''Return the tokens of a sentence including punctuation.
    >>> tokenize('Bob dropped the apple. Where is the apple?')
    ['Bob', 'dropped', 'the', 'apple', '.', 'Where', 'is', 'the', 'apple', '?']
    '''
    return [x.strip() for x in re.split('(\W+)?', sent) if x.strip()]

In [3]:
def parse_stories(lines, only_supporting=False):
    '''Parse stories provided in the bAbi tasks format
    If only_supporting is true, only the sentences
    that support the answer are kept.
    '''
    data = []
    story = []
    for line in lines:
        line = line.decode('utf-8').strip()
        nid, line = line.split(' ', 1)
        nid = int(nid)
        if nid == 1:
            story = []
        if '\t' in line:
            q, a, supporting = line.split('\t')
            q = tokenize(q)
            substory = None
            if only_supporting:
                # Only select the related substory
                supporting = map(int, supporting.split())
                substory = [story[i - 1] for i in supporting]
            else:
                # Provide all the substories
                substory = [x for x in story if x]
            data.append((substory, q, a))
            story.append('')
        else:
            sent = tokenize(line)
            story.append(sent)
    return data

In [4]:
def get_stories(f, only_supporting=False, max_length=None):
    '''Given a file name, read the file,
    retrieve the stories,
    and then convert the sentences into a single story.
    If max_length is supplied,
    any stories longer than max_length tokens will be discarded.
    '''
    data = parse_stories(f.readlines(), only_supporting=only_supporting)
    flatten = lambda data: reduce(lambda x, y: x + y, data)
    data = [(flatten(story), q, answer) for story, q, answer in data if not max_length or len(flatten(story)) < max_length]
    return data

In [5]:
def vectorize_stories(data, word_idx, story_maxlen, query_maxlen):
    X = []
    Xq = []
    Y = []
    for story, query, answer in data:
        x = [word_idx[w] for w in story]
        xq = [word_idx[w] for w in query]
        # let's not forget that index 0 is reserved
        y = np.zeros(len(word_idx) + 1)
        y[word_idx[answer]] = 1
        X.append(x)
        Xq.append(xq)
        Y.append(y)
    return (pad_sequences(X, maxlen=story_maxlen),
            pad_sequences(Xq, maxlen=query_maxlen), np.array(Y))

In [6]:
try:
    path = get_file('babi-tasks-v1-2.tar.gz', origin='https://s3.amazonaws.com/text-datasets/babi_tasks_1-20_v1-2.tar.gz')
except:
    print('Error downloading dataset, please download it manually:\n'
          '$ wget http://www.thespermwhale.com/jaseweston/babi/tasks_1-20_v1-2.tar.gz\n'
          '$ mv tasks_1-20_v1-2.tar.gz ~/.keras/datasets/babi-tasks-v1-2.tar.gz')
    raise
tar = tarfile.open(path)

challenges = {
    # QA1 with 10,000 samples
    'single_supporting_fact_10k': 'tasks_1-20_v1-2/en-10k/qa1_single-supporting-fact_{}.txt',
    # QA2 with 10,000 samples
    'two_supporting_facts_10k': 'tasks_1-20_v1-2/en-10k/qa2_two-supporting-facts_{}.txt',
}
challenge_type = 'single_supporting_fact_10k'
challenge = challenges[challenge_type]


In [7]:
print('Extracting stories for the challenge:', challenge_type)
train_stories = get_stories(tar.extractfile(challenge.format('train')))
test_stories = get_stories(tar.extractfile(challenge.format('test')))

vocab = set()
for story, q, answer in train_stories + test_stories:
    vocab |= set(story + q + [answer])
vocab = sorted(vocab)

Extracting stories for the challenge: single_supporting_fact_10k


/home/ratan/anaconda3/lib/python3.6/re.py:212: FutureWarning: split() requires a non-empty pattern match.
  return _compile(pattern, flags).split(string, maxsplit)


In [8]:
# Reserve 0 for masking via pad_sequences
vocab_size = len(vocab) + 1
story_maxlen = max(map(len, (x for x, _, _ in train_stories + test_stories)))
query_maxlen = max(map(len, (x for _, x, _ in train_stories + test_stories)))

print('-')
print('Vocab size:', vocab_size, 'unique words')
print('Story max length:', story_maxlen, 'words')
print('Query max length:', query_maxlen, 'words')
print('Number of training stories:', len(train_stories))
print('Number of test stories:', len(test_stories))
print('-')
print('Here\'s what a "story" tuple looks like (input, query, answer):')
print(train_stories[0])
print('-')
print('Vectorizing the word sequences...')

-
Vocab size: 22 unique words
Story max length: 68 words
Query max length: 4 words
Number of training stories: 10000
Number of test stories: 1000
-
Here's what a "story" tuple looks like (input, query, answer):
(['Mary', 'moved', 'to', 'the', 'bathroom', '.', 'John', 'went', 'to', 'the', 'hallway', '.'], ['Where', 'is', 'Mary', '?'], 'bathroom')
-
Vectorizing the word sequences...


In [9]:
word_idx = dict((c, i + 1) for i, c in enumerate(vocab))
inputs_train, queries_train, answers_train = vectorize_stories(train_stories,
                                                               word_idx,
                                                               story_maxlen,
                                                               query_maxlen)
inputs_test, queries_test, answers_test = vectorize_stories(test_stories,
                                                            word_idx,
                                                            story_maxlen,
                                                            query_maxlen)

print('-')
print('inputs: integer tensor of shape (samples, max_length)')
print('inputs_train shape:', inputs_train.shape)
print('inputs_test shape:', inputs_test.shape)
print('-')
print('queries: integer tensor of shape (samples, max_length)')
print('queries_train shape:', queries_train.shape)
print('queries_test shape:', queries_test.shape)
print('-')
print('answers: binary (1 or 0) tensor of shape (samples, vocab_size)')
print('answers_train shape:', answers_train.shape)
print('answers_test shape:', answers_test.shape)
print('-')
print('Compiling...')

-
inputs: integer tensor of shape (samples, max_length)
inputs_train shape: (10000, 68)
inputs_test shape: (1000, 68)
-
queries: integer tensor of shape (samples, max_length)
queries_train shape: (10000, 4)
queries_test shape: (1000, 4)
-
answers: binary (1 or 0) tensor of shape (samples, vocab_size)
answers_train shape: (10000, 22)
answers_test shape: (1000, 22)
-
Compiling...


In [10]:
# placeholders
input_sequence = Input((story_maxlen,))
question = Input((query_maxlen,))

# encoders
# embed the input sequence into a sequence of vectors
input_encoder_m = Sequential()
input_encoder_m.add(Embedding(input_dim=vocab_size,
                              output_dim=64))
input_encoder_m.add(Dropout(0.3))
# output: (samples, story_maxlen, embedding_dim)

# embed the input into a sequence of vectors of size query_maxlen
input_encoder_c = Sequential()
input_encoder_c.add(Embedding(input_dim=vocab_size,
                              output_dim=query_maxlen))
input_encoder_c.add(Dropout(0.3))
# output: (samples, story_maxlen, query_maxlen)

# embed the question into a sequence of vectors
question_encoder = Sequential()
question_encoder.add(Embedding(input_dim=vocab_size,
                               output_dim=64,
                               input_length=query_maxlen))
question_encoder.add(Dropout(0.3))
# output: (samples, query_maxlen, embedding_dim)

# encode input sequence and questions (which are indices)
# to sequences of dense vectors
input_encoded_m = input_encoder_m(input_sequence)
input_encoded_c = input_encoder_c(input_sequence)
question_encoded = question_encoder(question)

In [11]:
# compute a 'match' between the first input vector sequence
# and the question vector sequence
# shape: `(samples, story_maxlen, query_maxlen)`
match = dot([input_encoded_m, question_encoded], axes=(2, 2))
match = Activation('softmax')(match)

# add the match matrix with the second input vector sequence
response = add([match, input_encoded_c])  # (samples, story_maxlen, query_maxlen)
response = Permute((2, 1))(response)  # (samples, query_maxlen, story_maxlen)

# concatenate the match matrix with the question vector sequence
answer = concatenate([response, question_encoded])

# the original paper uses a matrix multiplication for this reduction step.
# we choose to use a RNN instead.
answer = LSTM(32)(answer)  # (samples, 32)

# one regularization layer -- more would probably be needed.
answer = Dropout(0.3)(answer)
answer = Dense(vocab_size)(answer)  # (samples, vocab_size)
# we output a probability distribution over the vocabulary
answer = Activation('softmax')(answer)

# build the final model
model = Model([input_sequence, question], answer)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',
              metrics=['accuracy'])

# train
model.fit([inputs_train, queries_train], answers_train,
          batch_size=32,
          epochs=400,
          validation_data=([inputs_test, queries_test], answers_test))

Train on 10000 samples, validate on 1000 samples
Epoch 1/400
10000/10000 [==============================] - 3s 340us/step - loss: 1.9414 - acc: 0.1671 - val_loss: 1.7983 - val_acc: 0.2540
Epoch 2/400
10000/10000 [==============================] - 2s 238us/step - loss: 1.6920 - acc: 0.2793 - val_loss: 1.5620 - val_acc: 0.4040
Epoch 3/400
10000/10000 [==============================] - 2s 236us/step - loss: 1.5618 - acc: 0.3721 - val_loss: 1.4926 - val_acc: 0.4050
Epoch 4/400
10000/10000 [==============================] - 2s 226us/step - loss: 1.5145 - acc: 0.3990 - val_loss: 1.4955 - val_acc: 0.3940
Epoch 5/400
10000/10000 [==============================] - 2s 223us/step - loss: 1.5005 - acc: 0.3970 - val_loss: 1.4843 - val_acc: 0.4130
Epoch 6/400
10000/10000 [==============================] - 2s 221us/step - loss: 1.4774 - acc: 0.4198 - val_loss: 1.4613 - val_acc: 0.4500
Epoch 7/400
10000/10000 [==============================] - 2s 218us/step - loss: 1.4368 - acc: 0.4459 - val_loss: 1.4

10000/10000 [==============================] - 2s 222us/step - loss: 0.5127 - acc: 0.8044 - val_loss: 0.5298 - val_acc: 0.7760
Epoch 60/400
10000/10000 [==============================] - 2s 225us/step - loss: 0.4849 - acc: 0.8133 - val_loss: 0.5274 - val_acc: 0.7930
Epoch 61/400
10000/10000 [==============================] - 2s 226us/step - loss: 0.4518 - acc: 0.8271 - val_loss: 0.4744 - val_acc: 0.7970
Epoch 62/400
10000/10000 [==============================] - 2s 234us/step - loss: 0.4330 - acc: 0.8353 - val_loss: 0.4439 - val_acc: 0.8120
Epoch 63/400
10000/10000 [==============================] - 2s 228us/step - loss: 0.4113 - acc: 0.8464 - val_loss: 0.4295 - val_acc: 0.8210
Epoch 64/400
10000/10000 [==============================] - 2s 225us/step - loss: 0.3912 - acc: 0.8541 - val_loss: 0.4278 - val_acc: 0.8240
Epoch 65/400
10000/10000 [==============================] - 2s 222us/step - loss: 0.3745 - acc: 0.8583 - val_loss: 0.4122 - val_acc: 0.8310
Epoch 66/400
10000/10000 [=======

10000/10000 [==============================] - 2s 219us/step - loss: 0.0858 - acc: 0.9703 - val_loss: 0.1613 - val_acc: 0.9550
Epoch 118/400
10000/10000 [==============================] - 2s 220us/step - loss: 0.0907 - acc: 0.9704 - val_loss: 0.1601 - val_acc: 0.9530
Epoch 119/400
10000/10000 [==============================] - 2s 222us/step - loss: 0.0863 - acc: 0.9689 - val_loss: 0.1975 - val_acc: 0.9410
Epoch 120/400
10000/10000 [==============================] - 2s 220us/step - loss: 0.0825 - acc: 0.9715 - val_loss: 0.1803 - val_acc: 0.9510
Epoch 121/400
10000/10000 [==============================] - 2s 219us/step - loss: 0.0846 - acc: 0.9716 - val_loss: 0.2068 - val_acc: 0.9440
Epoch 122/400
10000/10000 [==============================] - 2s 217us/step - loss: 0.0778 - acc: 0.9721 - val_loss: 0.1753 - val_acc: 0.9500
Epoch 123/400
10000/10000 [==============================] - 2s 240us/step - loss: 0.0749 - acc: 0.9746 - val_loss: 0.1736 - val_acc: 0.9460
Epoch 124/400
10000/10000 [

10000/10000 [==============================] - 2s 214us/step - loss: 0.0323 - acc: 0.9907 - val_loss: 0.1493 - val_acc: 0.9640
Epoch 176/400
10000/10000 [==============================] - 2s 241us/step - loss: 0.0255 - acc: 0.9916 - val_loss: 0.1314 - val_acc: 0.9690
Epoch 177/400
10000/10000 [==============================] - 2s 245us/step - loss: 0.0314 - acc: 0.9907 - val_loss: 0.1534 - val_acc: 0.9650
Epoch 178/400
10000/10000 [==============================] - 2s 227us/step - loss: 0.0289 - acc: 0.9913 - val_loss: 0.1690 - val_acc: 0.9700
Epoch 179/400
10000/10000 [==============================] - 2s 216us/step - loss: 0.0348 - acc: 0.9884 - val_loss: 0.1569 - val_acc: 0.9690
Epoch 180/400
10000/10000 [==============================] - 2s 217us/step - loss: 0.0334 - acc: 0.9898 - val_loss: 0.1525 - val_acc: 0.9660
Epoch 181/400
10000/10000 [==============================] - 2s 217us/step - loss: 0.0363 - acc: 0.9893 - val_loss: 0.1252 - val_acc: 0.9740
Epoch 182/400
10000/10000 [

10000/10000 [==============================] - 2s 212us/step - loss: 0.0194 - acc: 0.9957 - val_loss: 0.1221 - val_acc: 0.9720
Epoch 234/400
10000/10000 [==============================] - 2s 213us/step - loss: 0.0271 - acc: 0.9930 - val_loss: 0.0969 - val_acc: 0.9770
Epoch 235/400
10000/10000 [==============================] - 2s 212us/step - loss: 0.0217 - acc: 0.9945 - val_loss: 0.1442 - val_acc: 0.9730
Epoch 236/400
10000/10000 [==============================] - 2s 212us/step - loss: 0.0193 - acc: 0.9943 - val_loss: 0.1491 - val_acc: 0.9690
Epoch 237/400
10000/10000 [==============================] - 2s 213us/step - loss: 0.0215 - acc: 0.9934 - val_loss: 0.1289 - val_acc: 0.9760
Epoch 238/400
10000/10000 [==============================] - 2s 212us/step - loss: 0.0272 - acc: 0.9931 - val_loss: 0.1442 - val_acc: 0.9710
Epoch 239/400
10000/10000 [==============================] - 2s 215us/step - loss: 0.0228 - acc: 0.9942 - val_loss: 0.1055 - val_acc: 0.9750
Epoch 240/400
10000/10000 [

10000/10000 [==============================] - 2s 214us/step - loss: 0.0191 - acc: 0.9949 - val_loss: 0.1343 - val_acc: 0.9730
Epoch 292/400
10000/10000 [==============================] - 2s 216us/step - loss: 0.0236 - acc: 0.9940 - val_loss: 0.1509 - val_acc: 0.9680
Epoch 293/400
10000/10000 [==============================] - 2s 214us/step - loss: 0.0301 - acc: 0.9931 - val_loss: 0.1057 - val_acc: 0.9740
Epoch 294/400
10000/10000 [==============================] - 2s 214us/step - loss: 0.0179 - acc: 0.9944 - val_loss: 0.1456 - val_acc: 0.9730
Epoch 295/400
10000/10000 [==============================] - 2s 212us/step - loss: 0.0327 - acc: 0.9937 - val_loss: 0.1067 - val_acc: 0.9760
Epoch 296/400
10000/10000 [==============================] - 2s 217us/step - loss: 0.0259 - acc: 0.9939 - val_loss: 0.1205 - val_acc: 0.9800
Epoch 297/400
10000/10000 [==============================] - 2s 213us/step - loss: 0.0223 - acc: 0.9947 - val_loss: 0.1369 - val_acc: 0.9750
Epoch 298/400
10000/10000 [

10000/10000 [==============================] - 2s 209us/step - loss: 0.0221 - acc: 0.9957 - val_loss: 0.0991 - val_acc: 0.9800
Epoch 350/400
10000/10000 [==============================] - 2s 207us/step - loss: 0.0193 - acc: 0.9956 - val_loss: 0.1104 - val_acc: 0.9810
Epoch 351/400
10000/10000 [==============================] - 2s 208us/step - loss: 0.0273 - acc: 0.9947 - val_loss: 0.1273 - val_acc: 0.9760
Epoch 352/400
10000/10000 [==============================] - 2s 208us/step - loss: 0.0229 - acc: 0.9945 - val_loss: 0.1442 - val_acc: 0.9720
Epoch 353/400
10000/10000 [==============================] - 2s 208us/step - loss: 0.0233 - acc: 0.9953 - val_loss: 0.1058 - val_acc: 0.9760
Epoch 354/400
10000/10000 [==============================] - 2s 207us/step - loss: 0.0165 - acc: 0.9960 - val_loss: 0.0891 - val_acc: 0.9800
Epoch 355/400
10000/10000 [==============================] - 2s 208us/step - loss: 0.0210 - acc: 0.9950 - val_loss: 0.0908 - val_acc: 0.9800
Epoch 356/400
10000/10000 [

In [12]:
answers_pred = model.predict([inputs_test,queries_test])

In [13]:
from sklearn.metrics import accuracy_score
accuracy_score(answers_test, answers_pred.round())

0.984